# Usign NLP to classify PDF docs 
(Six labels - Nota, DOC, BOLETO ,RPAs, RECIBOS and OUTROS)

In [1]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

import pandas, numpy
import pandas as pd 
import numpy as np

from nltk.corpus import webtext 
from nltk.collocations import BigramCollocationFinder 
from nltk.metrics import BigramAssocMeasures 
from random import random
from collections           import defaultdict
from gensim.models.phrases import Phrases, Phraser

In [2]:
df4= pd.DataFrame(columns = ['body','label','name','label1',"array"])
#df3.info()
df4=df4[df4.label == "ALL"]
df4=df4.reindex()
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    0 non-null      object
 1   label   0 non-null      object
 2   name    0 non-null      object
 3   label1  0 non-null      object
 4   array   0 non-null      object
dtypes: object(5)
memory usage: 0.0+ bytes


In [3]:
from os import listdir
from os.path import isfile, join
mypath = 'C:/AI/krypton-01/texto-01'
onlyfile1 = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(len(onlyfile1))

332


In [4]:
for file1 in onlyfile1:
    #print(file1[-3:])
    #print(file1)
    #print(file1[:-4])
    if file1[-3:]=="txt":
        print(file1[:-4])
        text1 = open('C:/AI/krypton-01/texto-01/'+file1, "r+",encoding = 'utf-8-sig').read()
        #print(text1)
        df2 = pd.DataFrame([[text1, "",file1[:-4],"",""]], columns = ['body','label','name','label1',"array"])
        #print(df2)
        df4=df4.append(df2)
        df4.reset_index(inplace = True, drop = True)
df4['body1']=df4['body']

2018-01-26_14-59_1231485
2018-01-26_14-59_1448667
2018-01-26_14-59_1697049
2019-10-14_09-30-58-1571056258_E_1488_6110909
2019-10-18_17-38_CP_I_986__8603450
2019-11-05_08-51-38-1572954698_E_1531_3116338
2019-11-05_08-51-38-1572954698_E_1531_8914399
2019-11-12_17-33_CP_E_1921__1349845
2019-12-04_10-46-18-1575467178_E_1531_6479943
2019-12-04_10-46-20-1575467180_E_1531_3294177
2019-12-04_11-07-14-1575468434_E_1531_4244244
2019-12-04_11-07-15-1575468435_E_1531_2134247
2019-12-04_11-07-15-1575468435_E_1531_4595260
2019-12-04_11-07-18-1575468438_E_1531_5476972
2019-12-17_16-33_CP_E_1921__2239244
2019-12-23_07-31-51-1577097111_E_1531_8440154
2019-12-23_07-31-54-1577097114_E_1531_4404655
2019-12-23_07-36-28-1577097388_E_1531_7985929
2019-12-23_07-36-29-1577097389_E_1531_9587191
2019-12-23_07-36-31-1577097391_E_1531_2632865
2019-12-23_07-36-32-1577097392_E_1531_6135714
2019-12-23_07-36-32-1577097392_E_1531_7600562
2020-01-23_12-54_CP_I_1044__3653671
2020-02-03_10-52-58-1580737978_E_1531_5650321


2021-02-23_11-02-33-1614088953_E_270_7037556
2021-02-23_11-30-29-1614090629_E_2017_3947762
2021-02-23_11-30-29-1614090629_E_2017_6528021
2021-02-23_11-30-29-1614090629_E_2017_8161644
2021-02-23_11-30-30-1614090630_E_2017_7702402
2021-02-23_11-30-30-1614090630_E_2017_9290125
2021-02-23_11-30-31-1614090631_E_2017_2610392
2021-02-23_11-30-31-1614090631_E_2017_3867918
2021-02-23_11-30-31-1614090631_E_2017_4693951
2021-02-23_11-30-32-1614090632_E_2017_7692163
2021-02-23_11-30-34-1614090634_E_2017_3023466-0
2021-02-23_11-30-34-1614090634_E_2017_3023466-1
2021-02-23_11-31-11-1614090671_E_2017_2683259
2021-02-23_11-31-11-1614090671_E_2017_6881835
2021-02-23_11-31-11-1614090671_E_2017_6926854
2021-02-23_11-31-12-1614090672_E_2017_6761746
2021-02-23_11-31-13-1614090673_E_2017_4443349
2021-02-23_11-31-13-1614090673_E_2017_4614002
2021-02-23_11-31-13-1614090673_E_2017_6145712
2021-02-23_11-31-14-1614090674_E_2017_5799212
2021-02-23_11-31-14-1614090674_E_2017_6399952
2021-02-23_11-31-14-1614090674_

In [5]:
# create function for removing special caracters 0
import html
import re
def clean(s):
    s = html.unescape(s)
    s = s.replace("\\n"," ")
    s = s.replace("_NEG","")
    s = s.replace("_NEGFIRST", "")
    s = re.sub(r"_", "", s)
    s = re.sub(r"#", "", s)
    s = re.sub(r"\*", "", s)
    s = re.sub(r"\'s", "", s)
    s = re.sub(r"\'m", " am", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'d", " would", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r",", "", s)
    s = re.sub(r"!", " !", s)
    s = re.sub(r"\(", "", s)
    s = re.sub(r"\)", "", s)
    s = re.sub(r"\?", " ?", s)
    s = re.sub(r'[^\x00-\x7F]',' ', s)
    s = re.sub(r'[^\w\s]',' ',s)
    s = re.sub("\d+", " ", s)
    s = re.sub(r"\s{2,}", " ", s)
    s = s.rstrip(',|.|;|:|\'|\"')
    s = s.lstrip('\'|\"')
    s = s.upper()
    return str(s)

In [6]:
# create function for removing special caracters 1
import html
import re
def clean1(s):
    stopwords=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','X','Z','W','Y','0','1','2','3','4','5','6','7','8','9']
    #len7=len(y1)
    words = re.split(r'\W+', s)
    body1=""
    for i1 in words:
        for i in stopwords:
            if str(i1)==i:
                    #print(y1['column1'][i],i1)
                    i1=""
        body1 = body1+' '+i1
        #print(y1['column1'][i],y1['column2'][i],'Tamanho = 1',s)
        #s = s.replace(y1['column1'][i], "")
        #print('removi tamanho =1',s)
    s=body1
    return str(s)

In [7]:
# Convert list into String
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [8]:
# apply clean function to body
for i in range(len(df4)):
      df4["body"][i]= clean(str(df4["body"][i]))
      print(i,df4['name'][i])

0 2018-01-26_14-59_1231485
1 2018-01-26_14-59_1448667
2 2018-01-26_14-59_1697049
3 2019-10-14_09-30-58-1571056258_E_1488_6110909
4 2019-10-18_17-38_CP_I_986__8603450
5 2019-11-05_08-51-38-1572954698_E_1531_3116338
6 2019-11-05_08-51-38-1572954698_E_1531_8914399
7 2019-11-12_17-33_CP_E_1921__1349845
8 2019-12-04_10-46-18-1575467178_E_1531_6479943
9 2019-12-04_10-46-20-1575467180_E_1531_3294177
10 2019-12-04_11-07-14-1575468434_E_1531_4244244
11 2019-12-04_11-07-15-1575468435_E_1531_2134247
12 2019-12-04_11-07-15-1575468435_E_1531_4595260
13 2019-12-04_11-07-18-1575468438_E_1531_5476972
14 2019-12-17_16-33_CP_E_1921__2239244
15 2019-12-23_07-31-51-1577097111_E_1531_8440154
16 2019-12-23_07-31-54-1577097114_E_1531_4404655
17 2019-12-23_07-36-28-1577097388_E_1531_7985929
18 2019-12-23_07-36-29-1577097389_E_1531_9587191
19 2019-12-23_07-36-31-1577097391_E_1531_2632865
20 2019-12-23_07-36-32-1577097392_E_1531_6135714
21 2019-12-23_07-36-32-1577097392_E_1531_7600562
22 2020-01-23_12-54_CP_I_1

262 2021-02-24_08-38_CP_I_1053__2125733
263 2021-02-24_09-33-27-1614170007_CP_I_978_6832306
264 2021-02-24_10-40-13-1614174013_E_1205_2203582-0
265 2021-02-24_10-40-13-1614174013_E_1205_2203582-1
266 2021-02-24_10-40-13-1614174013_E_1205_8882487
267 2021-02-24_10-40-14-1614174014_E_1205_1840543-0
268 2021-02-24_10-40-14-1614174014_E_1205_1840543-1
269 2021-02-24_10-40-14-1614174014_E_1205_5100396
270 2021-02-24_10-40-14-1614174014_E_1205_6844501-0
271 2021-02-24_10-40-14-1614174014_E_1205_6844501-1
272 2021-02-24_10-40-14-1614174014_E_1205_7740837-0
273 2021-02-24_10-40-14-1614174014_E_1205_7740837-1
274 2021-02-24_10-40-14-1614174014_E_1205_8270040-0
275 2021-02-24_10-40-14-1614174014_E_1205_8270040-1
276 2021-02-24_10-40-14-1614174014_E_1205_9405719-0
277 2021-02-24_10-40-14-1614174014_E_1205_9405719-1
278 2021-02-24_10-40-15-1614174015_E_1205_1585272
279 2021-02-24_10-40-15-1614174015_E_1205_2414649
280 2021-02-24_10-40-15-1614174015_E_1205_5233192
281 2021-02-24_10-40-15-1614174015

In [9]:
# apply clean function to body
len5 = len(df4)
for i in range(len5):
    #print(i)
    #zz= clean1(str(df3["body"][i]))
    df4["body"][i]= clean1(str(df4["body"][i]))
    #print(i,df3['name'][i])
    #print(i,zz)

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

# apply clean function to body
len5 = len(df4)
for i in range(len5):
    print(i)
    #zz= clean1(str(df3["body"][i]))
    text = df4["body"][i]
    text_tokens = word_tokenize(text)
    textx = [word for word in text_tokens if not word in stopwords.words('portuguese')]
    df4["body"][i]=listToString(textx)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ziulc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
len5=len(df4)
df4.info()
for i in range(len5):
    text = str(df4['body'][i])
    textx = str(df4['label'][i])
    text1 = text.encode('utf-8')
    text2 = textx.encode('utf-8')
    df4['body'][i] = text
    #df3['label'][i]= textx

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    332 non-null    object
 1   label   332 non-null    object
 2   name    332 non-null    object
 3   label1  332 non-null    object
 4   array   332 non-null    object
 5   body1   332 non-null    object
dtypes: object(6)
memory usage: 15.7+ KB


In [12]:
# Create syntetic variables
df4['sint1']=0
df4['sint2']=0
df4['sint3']=0
df4['sint4']=0
df4['sint5']=0
df4['sint6']=0
from nltk.tokenize import word_tokenize
len5 = len(df4)
for i in range(len5):
    #print(i)
    #zz= clean1(str(df3["body"][i]))
    text = df4["body"][i]
    stopwords1=['BANCO','FATURA','PROVANTE','VENCIMENTO','DEBIADA']
    stopwords2=['NFS','NOTA','FISCAL','PREFEITURA','MUNICIPAL']
    stopwords3=['RECIBO','RECEBEMOS','RECEBI','ALUGUEL']
    stopwords4=['RPA','NOMO','AUT','EMITENTE']
    stopwords5=['BANCO','PROVANTE','BOT','DEBIADA','VENCIMENTO']
    stopwords6=['CONTRATO','AUXILIAR','DOCUMENTO','DANFPS']
    words = re.split(r'\W+', text)
    body1=""
    for i1 in words:
        
        for ix1 in stopwords1:
            if str(i1)==ix1:
                print(i1)
                df4['sint1'][i]=df4['sint3'][i]+1
        for ix2 in stopwords2:
            if str(i1)==ix2:
                print(i1)
                df4['sint2'][i]=df4['sint3'][i]+1
        for ix3 in stopwords3:
            if str(i1)==ix3:
                print(i1)
                df4['sint3'][i]=df4['sint3'][i]+1
        for ix3 in stopwords4:
            if str(i1)==ix3:
                print(i1)
                df4['sint4'][i]=df4['sint4'][i]+1
        for ix3 in stopwords5:
            if str(i1)==ix3:
                print(i1)
                df4['sint5'][i]=df4['sint5'][i]+1
        for ix3 in stopwords6:
            if str(i1)==ix3:
                print(i1)
                df4['sint6'][i]=df4['sint6'][i]+1
#for i in range(len5):
#    ttotal=df3['sint1'][i]+df3['sint2'][i]+df3['sint3'][i]
#    if ttotal!=0:
#        df3['sint1'][i]=df3['sint1'][i]/ttotal
#        df3['sint2'][i]=df3['sint2'][i]/ttotal
#        df3['sint3'][i]=df3['sint3'][i]/ttotal 

NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO
AUT
NOMO
RPA
RPA
RECIBO
RECIBO
AUT
NOMO
RPA
RECEBI
RECIBO
RECEBI
RPA
NFS
NOTA
FISCAL
MUNICIPAL
NFS
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO
AUT
NOMO
RPA
RPA
RECIBO
RECIBO
AUT
NOMO
RPA
RECEBI
RECIBO
RECEBI
RPA
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
PREFEITURA
MUNICIPAL
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NFS
MUNICIPAL
NOTA
FISCAL
NOTA
FISCAL
DOCUMENTO
FISCAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A valu


MUNICIPAL
NOTA
FISCAL
MUNICIPAL
NOTA
NOTA
FISCAL
AUT
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NOTA
MUNICIPAL
FISCAL
NOTA
FISCAL
NOTA
FISCAL
DOCUMENTO
DOCUMENTO
NOTA
FISCAL
NOTA
FISCAL
NOTA
FISCAL
NOTA
FISCAL
PREFEITURA
MUNICIPAL
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NFS
MUNICIPAL
NOTA
FISCAL
NOTA
FISCAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NOTA
FISCAL
FATURA
NOTA
NOTA
PREFEITURA
NOTA
FISCAL
NOTA
FISCAL
MUNICIPAL
EMITENTE
FATURA
FATURA
NOTA
PREFEITURA
NOTA
NOTA
NOTA
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
PREFEITURA
MUNICIPAL
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NFS
MUNICIPAL
NOTA
FISCAL
NOTA
FISCAL
DOCUMENTO
FISCAL
NOTA
FISCAL
MUNICIPAL
NOTA
NOTA
FISCAL
AUT
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NOTA
FISCAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



AUT
NOMO
RPA
RPA
RECIBO
RECIBO
AUT
NOMO
RPA
RECEBI
RECIBO
RECEBI
RPA
RECIBO
AUT
NOMO
RPA
RECIBO
AUT
NOMO
RPA
RPA
RECIBO
RECEBI
RECIBO
RECEBI
RPA
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO
AUT
NOMO
RPA
RPA
RECIBO
RECIBO
AUT
NOMO
RPA
RECEBI
RECIBO
RECEBI
RPA
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO
AUT
NOMO
RPA
RPA
RECIBO
RECIBO
AUT
NOMO
RPA
RECEBI
RECIBO
RECEBI
RPA
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
CONTRATO
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
CONTRATO
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
NOTA
MUNICIPAL
MUNICIPAL
NOTA
NFS
VENCIMENTO
VENCIMENTO
NFS
NFS
NFS


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO
AUT
NOMO
RPA
RPA
RECIBO
RECIBO
AUT
NOMO
RPA
RECEBI
RECIBO
RECEBI
RPA
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
PREFEITURA
NOTA
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
PREFEITURA
MUNICIPAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPA

NOTA
NFS
VENCIMENTO
VENCIMENTO
NFS
NFS
NFS
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
CONTRATO
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
NOTA
FISCAL
NOTA
PREFEITURA
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
NOTA
NOTA
MUNICIPAL
MUNICIPAL
NOTA
NOTA
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
CONTRATO
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
PREFEITURA
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
NOTA
NFS
VENCIMENTO
VENCIMENTO
NFS
NFS
NFS
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
PREFEITURA
MUNICIPAL
RECIBO
MUNICIPAL
RECIBO
VENCIMENTO
VENCIMENTO
VENCIMENTO
VENCIMENTO
MUNICIPAL
DOCUMENTO
RECIBO
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIP

VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
MUNICIPAL
NOTA
FISCAL
FATURA
NOTA
NOTA
NOTA
FISCAL
NOTA
FISCAL
PREFEITURA
MUNICIPAL
NOTA
NOTA
FATURA
FATURA
NOTA
FISCAL
PREFEITURA
NOTA
NOTA
NOTA
NFS
NOTA
FISCAL
NFS
NOTA
FISCAL
MUNICIPAL
MUNICIPAL
MUNICIPAL
DOCUMENTO
FISCAL
PREFEITURA
MUNICIPAL
RECEBEMOS
NOTA
FISCAL
DOCUMENTO
NOTA
FATURA
NOTA
MUNICIPAL
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NOTA
FISCAL
FATURA
NOTA
FISCAL
NOTA
NOTA
PREFEITURA
MUNICIPAL
DOCUMENTO
NOTA
FATURA
FATURA
NOTA
FISCAL
PREFEITURA
NOTA
NOTA
NOTA
FISCAL
PREFEITURA
MUNICIPAL
NOTA
NFS
MUNICIPAL
MUNICIPAL
VENCIMENTO
VENCIMENTO
NOTA
NFS
DOCUMENTO
DOCUMENTO
FISCAL
NFS
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
BANCO
BANCO
BANCO
BANCO
NFS
DOCUMENTO
PREFEITURA
NOTA
NOTA
RECIBO
MUNICIPAL
FISCAL
VENCIMENTO
VENCIMENTO
NOTA
NOTA
FISCAL
NOTA
PREFEITURA
MUNICIPAL
NOTA
FISCAL
NFS
MUNICIPAL
MUNICIPAL
NFS
NFS
NFS
VENCIMENTO
VENCIMENTO
NFS
PREFEITURA
BANCO
BANCO
RECIBO
VENCIMENTO
VENCIMENTO
DOCUMENTO
VENCIMENTO
VENCIMEN

In [13]:
#Versao melhorada
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import scipy.sparse as sp
from scipy.sparse import coo_matrix
tfidfconverter = TfidfVectorizer(max_features=200, min_df=5,max_df=0.7,stop_words=stopwords.words('portuguese'))


var0 = coo_matrix((df4['sint1']))
var1 = coo_matrix((df4['sint2']))
var2 = coo_matrix((df4['sint3']))
var3 = coo_matrix((df4['sint4']))
var4 = coo_matrix((df4['sint5']))
var5 = coo_matrix((df4['sint6']))

X6 = tfidfconverter.fit_transform(df4['body']).toarray()
combined_X6 = sp.hstack([var0.T,var1.T,var2.T,var3.T,var4.T,var5.T,X6], format='csr')

X2 = combined_X6

In [15]:
import pickle
filename = 'C:/AI/krypton-01/csv-doc/finalized_model.sav'
eclf_model = pickle.load(open(filename, 'rb'))

In [16]:
#lr_results2 = lr_model.predict(X2)
eclf_result2 = eclf_model.predict(X2)
#lr_results_prob1 = eclf_model.predict_proba(X2)

In [17]:
len3=len(df4)
df4['result1']=""
df4['nota_pro']=0.0
df4['doc_pro']=0.0
df4['recibo_pro']=0.0
df4['boleto_pro']=0.0
df4['rpa_pro']=0.0
df4['outros_pro']=0.0
for i in range(len3):
    df4['result1'][i]=eclf_result2[i]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
df4.to_csv('C:/AI/krypton-01/csv-doc/revisado8-3-01.csv')